# **The HandTracker class**

This notebook provides some documentation about ```HandTracker``` objects.

Run the following codeblocks to import the ```HandTracker``` class into this notebook.

In [1]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: c:\Users\Brad\Documents\Projects\Python\fivecarddraw


In [2]:
from fivecarddraw import HandTracker

## **Initialisation**

The ```HandTracker``` class is designed to be mediated by ```Dealer``` objects. Upon initialisation:
* It creates a ```Deck``` in the attribute ```Handtracker.DECK```, and shuffles the deck using ```DECK.Shuffle()```.
* It loads about 150kb of data from [lookup tables](../lookup-tables/) using ```HandTracker.LoadLookupTables()```, and creates three ```dict``` attributes containing data useful for evaluating hands, namely: ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS```, and ```HandTracker.DUPE_RANKS```.
* It creates two ```dict``` attributes that provide ciphers useful for encoding hands as an ```int```, for speedy hand evaluation, namely: ```HandTracker.DV``` and ```HandTracker.DP```.
* It creates two ```tuple``` attributes containing data for categorising hands, namely: ```HandTracker.CLASSES``` and ```HandTracker.BOUNDARIES```.
* It creates a ```dict``` that stores data about players hands as json in the attribute ```HandTracker.hands``` 

For more information about ```Deck``` objects, see the [deck.ipynb](deck.ipynb) notebook. For more information about ```Dealer``` objects see the [dealer.ipynb](dealer.ipynb) notebook.


In [19]:
tracker = HandTracker()
print(f"deck: {tracker.DECK}\n")
print(f"Amount of keys in FLUSH_RANKS: {len(tracker.FLUSH_RANKS)}")
print(f"Amount of keys in UNIQUE5_RANKS: {len(tracker.UNIQUE5_RANKS)}")
print(f"Amount of keys in DUPE_RANKS: {len(tracker.DUPE_RANKS)}\n")
print(f"Binary cipher: {tracker.DV}")
print(f"Prime cipher: {tracker.DP}\n")
print(f"Hand classifications: {tracker.CLASSES}")
print(f"Classification boundaries: {tracker.BOUNDARIES}")

deck: [K♠, 4♣, Q♡, 6♣, Q♠, A♢, 3♣, 2♣, K♡, 2♠, 3♡, 2♢, 10♠, A♣, 2♡, A♡, 8♠, Q♣, 6♠, 3♢, J♣, 8♢, 7♡, K♣, 6♡, 9♠, 9♢, 8♡, 5♣, Q♢, 4♠, J♡, 5♢, A♠, 7♣, 3♠, J♠, J♢, K♢, 10♣, 4♢, 6♢, 10♢, 10♡, 5♡, 9♡, 4♡, 8♣, 7♠, 9♣, 5♠, 7♢]

Amount of keys in FLUSH_RANKS: 1287
Amount of keys in UNIQUE5_RANKS: 1287
Amount of keys in DUPE_RANKS: 4888

Binary cipher: {'2': 1, '3': 2, '4': 4, '5': 8, '6': 16, '7': 32, '8': 64, '9': 128, 'T': 256, 'J': 512, 'Q': 1024, 'K': 2048, 'A': 4096}
Prime cipher: {'2': 2, '3': 3, '4': 5, '5': 7, '6': 11, '7': 13, '8': 17, '9': 19, 'T': 23, 'J': 29, 'Q': 31, 'K': 37, 'A': 41}

Hand classifications: ('High card', 'pair', 'two pair', 'three of a kind', 'straight', 'flush', 'full house', 'four of a kind', 'straight flush', 'royal flush')
Classification boundaries: (6186, 3326, 2468, 1610, 1600, 323, 167, 11, 2, 1)


### **LoadLookupTables()**

The folder [lookup-tables](../lookup-tables/) contains three different files which are tables of data that contain both numerical and categorical evaluations of every possible five card draw hand. The actual data in those files was scraped from an external website [here](http://suffe.cool/poker/7462.html). The purpose of the ```HandTracker.LoadLookupTables()``` method is to load this data into the ```HandTracker``` object so that it can be used to quickly evaluate hands. Furthermore, each file in the [lookup-tables](../lookup-tables/) folder is formatted in the same way. Each line can be interpreted as follows:

* The first two letters encode a categorical evaluation; either SF, 4K, FH, FF, SS, 3K, 2P, 1P or HC which stand for straight flush, four of a kind, full house, flush, straight, three of a kind, two pair, pair or high card respectively.
* The next five characters give the cards in a hand; either A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2.
* The remaining numbers give a numerical evaluation; 1 being the best and 7462 being the worst. These ordinals were determined through combinatorial analysis by [Kevin Suffecool](http://suffe.cool/poker/evaluator.html), the owner of the site where the data was scraped.

The ```HandTracker.LoadLookupTables()``` method converts each hand it reads from the [lookup-tables](../lookup-tables/) into an ```int``` that represents the hand irrepective of the order of cards in the hand. 

* For hands with no duplicates, the hand is converted to an ```int``` using the ```HandTracker.DV``` cipher. It depends upon binary arithmetic, with cards represented as powers of two and added together to encode the hand.
* For hands with duplicates, the hand is converted to an ```int``` using the ```HandTracker.DP``` cipher. It depends upon prime factorisation, with cards represented as prime numbers and multiplied together to encode the hand.

In [54]:
from functools import reduce
tracker = HandTracker()

flush_lookup = open("lookup-tables/flush lookup.txt", "r")
unique5_lookup = open("lookup-tables/unique five lookup.txt", "r")
dupe_lookup = open("lookup-tables/dupe lookup.txt", "r")

line = flush_lookup.readline()[:-1]
encoding = reduce(lambda x, y : x+y, map(lambda x : tracker.DV[line[int(x)]], '45678'))
print(f"Flush hand:\n{line}")
print(f"Binary encoding: {encoding}\n")

line = unique5_lookup.readline()[:-1]
encoding = reduce(lambda x, y : x+y, map(lambda x : tracker.DV[line[int(x)]], '45678'))
print(f"No-dupes, No-flush hand:\n{line}")
print(f"Binary encoding: {encoding}\n")

line = dupe_lookup.readline()[:-1]
encoding = reduce(lambda x, y : x*y, map(lambda x : tracker.DP[line[int(x)]], '45678'))
print(f"Dupes hand:\n{line}")
print(f"Prime encoding: {encoding}\n")

flush_lookup.close()
unique5_lookup.close()
dupe_lookup.close()

Flush hand:
SF  AKQJT  1
Binary encoding: 7936

No-dupes, No-flush hand:
SS  AKQJT  1600
Binary encoding: 7936

Dupes hand:
4K  AAAAK  11
Prime encoding: 104553157



## **Functionality**

### **Dealing Hands**

The recommnded way to deal cards to players is to use ```HandTracker.DealHands(names)```. 

In [18]:
tracker = HandTracker()
tracker.DealHands(["Brad", "Scott", "Daniel"])
print(f"Hands: {tracker.hands}\n")
print(f"Amount of remaining cards in deck: {tracker.DECK.CountRemaining()}")

Hands: {'Brad': {'cards': [A♢, K♡, A♡, 10♡, 9♣]}, 'Scott': {'cards': [6♠, 4♠, Q♢, 4♡, 9♡]}, 'Daniel': {'cards': [5♣, 8♡, 9♢, 9♠, 7♢]}}

Amount of remaining cards in deck: 37


### **Evaluating Hands**

The ```HandTracker.EvaluateHand()``` method is the recommended way to evaluate players hands. It does the following:

* It determines which of ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS``` or ```HandTracker.DUPE_RANKS``` attributes contain the rank of the hand.
* It converts the hand composition into an ```int``` that can be used as a key in the relevant attribute above, returning the numerical rank of the hand. There are 7462 different ranks in total, as calculated by [Kevin Suffecool](http://suffe.cool/poker/evaluator.html).
* It inteprets the numerical rank of the hand as a categorical rank using the ```HandTracker.CLASSES``` and ```HANDTRACKER.BOUNDARIES``` attributes.
* It stores the data in ```Handtracker.hands```.

In [36]:
tracker = HandTracker()
player = "Brad"
tracker.DealHands([player])
tracker.EvaluateHands()
print(f"Hand info: {tracker.hands[player]}")

Hand info: {'cards': [J♢, 10♠, 6♡, 10♢, 4♢], 'rank_n': 4361, 'rank_c': 'pair'}


#### **Locating Hand Rank**

The ```HandTracker.CheckFlush()``` and ```HandTracker.CheckUnique5()``` methods are used to determine which of ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS``` or ```HandTracker.DUPE_RANKS``` attributes contain the rank of the hand. This is done by:

* Converting the composition of the hand to an ```int``` by combining the ```Card.b``` values of each card. Bitwise-and is used for ```HandTracker.CheckFlush()```. Bitwise-or is used for ```HandTracker.CheckUnique5()```. 
* The ```bool``` values that ```HandTracker.CheckFlush()``` and ```HandTracker.CheckUnique5()``` return, are used to decide which of ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS``` or ```HandTracker.DUPE_RANKS``` attributes contain the rank of the hand.

for more information about ```Card.b``` is calculated see the [card.ipynb](card.ipynb) notebook.

In [58]:
from functools import reduce

tracker = HandTracker()
player = "Brad"
tracker.DealHands([player])
hand = tracker.hands[player]['cards']
and_encoding = bin(reduce(lambda x, y: x&y, map(lambda x : x.b, hand)))[2:].zfill(32)
or_encoding = bin(reduce(lambda x, y: x|y, map(lambda x : x.b, hand)))[2:].zfill(32)

print(f"Hand: {hand}")
print(f"{hand[0]} encoded: {bin(hand[0].b)[2:].zfill(32)}")
print(f"{hand[1]} encoded: {bin(hand[1].b)[2:].zfill(32)}")
print(f"{hand[2]} encoded: {bin(hand[2].b)[2:].zfill(32)}")
print(f"{hand[3]} encoded: {bin(hand[3].b)[2:].zfill(32)}")
print(f"{hand[4]} encoded: {bin(hand[4].b)[2:].zfill(32)}\n")
print(f"'and' encoding: {and_encoding}")
print(f"Flush mask: {bin(15 << 12)[2:].zfill(32)}")
print(f"Flush check: {tracker.CheckFlush(hand)}\n")
print(f"'or' encoding: {or_encoding}")
print(f"Unique5 mask: {bin((2**13 - 1) << 16)[2:].zfill(32)}")
print(f"Unique5 check: {tracker.CheckUnique5(hand)}\n")

print(f"Dupes check: {not tracker.CheckUnique5(hand) and not tracker.CheckFlush(hand)}")

Hand: [9♣, 9♢, 9♡, 10♣, Q♣]
9♣ encoded: 00000000100000001000011100010011
9♢ encoded: 00000000100000000100011100010011
9♡ encoded: 00000000100000000010011100010011
10♣ encoded: 00000001000000001000100000010111
Q♣ encoded: 00000100000000001000101000011111

'and' encoding: 00000000000000000000000000010011
Flush mask: 00000000000000001111000000000000
Flush check: False

'or' encoding: 00000101100000001110111100011111
Unique5 mask: 00011111111111110000000000000000
Unique5 check: False

Dupes check: True


#### **Reading Numerical Hand Rank**

Once the hand rank has been located, the hand needs to be converted into key to retreive the rank from either ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS```, or ```HandTracker.DUPE_RANKS``` respectively. 

* If the rank is in ```HandTracker.FLUSH_RANKS```, ```HandTracker.UNIQUE5_RANKS```, then ```HandTracker.ExtractSum(hand)``` should be used.
* If the rank is in ```HandTracker.DUPE_RANKS```, then ```HandTracker.ExtractProduct(hand)``` should be used.

In [89]:
tracker = HandTracker()
player = "Brad"
tracker.DealHands([player])
hand = tracker.hands[player]['cards']
flush = tracker.CheckFlush(hand)
unique5 = tracker.CheckUnique5(hand)

if flush:
    print(f"{hand} has flush.")
    encoding = tracker.ExtractSum(hand)
    print(f"hand encoding: {encoding}")
    print(f"hand rank: {tracker.FLUSH_RANKS[encoding]}")
elif unique5:
    print(f"{hand} has five unique values.")
    encoding = tracker.ExtractSum(hand)
    print(f"hand encoding: {encoding}")
    print(f"hand rank: {tracker.UNIQUE5_RANKS[encoding]}")
else:
    print(f"{hand} has dupe values.")
    encoding = tracker.ExtractProduct(hand)
    print(f"hand encoding: {encoding}")
    print(f"hand rank: {tracker.DUPE_RANKS[encoding]}")

[2♣, 4♢, 3♡, 5♢, 7♢] has five unique values.
hand encoding: 47
hand rank: 7462


#### **Categorising Hand Rank**

Using the numerical ranking of a hand, the categorical rank is found by filtering ```HandTracker.BOUNDARIES``` of numbers smaller than the rank, and then using the resultant length to index into ```HandTracker.CLASSES```. The boundaries were manually found from the [lookup-tables](../lookup-tables/).

In [96]:
hand_rank = 1000

tracker = HandTracker()
c = 10 - len(list(filter(lambda x : hand_rank >= x, tracker.BOUNDARIES)))
print(f"The hand ranked {hand_rank} is classified as a {tracker.CLASSES[c]}.")


The hand ranked 1000 is classified as a flush.


### **Collecting Cards**

The recommended way to collect cards from players is to use ```HandTracker.CollectHands()```, which clears ```HandTracker.hands``` and prepares ```HandTracker.DECK``` for a new round.

In [3]:
tracker = HandTracker()
tracker.DealHands(["Brad", "Scott", "Daniel"])
print(f"Amount of remaining cards in deck before collecting hands: {tracker.DECK.CountRemaining()}")
tracker.CollectHands()
print(f"Amount of remaining cards in deck after collecting hands: {tracker.DECK.CountRemaining()}")

Amount of remaining cards in deck before collecting hands: 37
Amount of remaining cards in deck after collecting hands: 52


### **Switching Cards**

The intended way to switch cards during the switching phase of a round of five card draw poker; is to first approve the selection of cards to be discarded using ```HandTracker.ApproveDiscards(name, discards)```, and once that returns ```True```, proceed to edit the hand using ```HandTracker.EditHand(name, discards)```. 

In [6]:
player = "Brad"
tracker = HandTracker()
tracker.DealHands([player])
print(f"{player}'s hand: {tracker.hands}\n")
discards = tracker.hands[player]["cards"][:3]
approval = tracker.ApproveDiscards(player, discards)
print(f"{player} wants to switch: {discards}")
print(f"Approval : {approval}\n")
if approval:
    tracker.EditHand(player, discards)
    print(f"{player}'s new hand: {tracker.hands}")

Brad's hand: {'Brad': {'cards': [5♢, 6♡, 9♡, A♣, K♡]}}

Brad wants to switch: [5♢, 6♡, 9♡]
Approval : True

Brad's new hand: {'Brad': {'cards': [A♣, K♡, 2♠, Q♣, 7♠]}}


## **Demos**

The ```HandTracker.DemoDeal()``` method can be used to create a random hand without the implict tracking of ```Card``` objects. The ```HandTracker.DemoEvaluate(hand)``` method can be used to evaluate a hand that isn't being tracked.  These methods are purely for demonstration purposes and are never called in [fivecarddraw.py](../fivecarddraw.py).

In [97]:
tracker = HandTracker()
hand = tracker.DemoDeal()
print(f"Random hand: {hand}")
print(f"Evaluation: {tracker.DemoEvaluate(hand)}")

Random hand: [7♣, 9♣, 5♣, Q♡, 4♠]
Evaluation: {'rank_c': 'High card', 'rank_n': 7166}
